In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
# Matplotlib and associated plotting modules
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
import sklearn
from sklearn.cluster import KMeans
import folium # map rendering library

print(f"Matplotlib version: {matplotlib.__version__}")
print(f"Numpy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Requests version: {requests.__version__}")
print(f"sklearn version: {sklearn.__version__}")
print(f"folium version: {folium.__version__}")

Matplotlib version: 3.4.2
Numpy version: 1.21.0
Pandas version: 1.2.5
Requests version: 2.15.1
sklearn version: 0.24.2
folium version: 0.12.1


## Toronto, Ontario neighborhood data

In [2]:
data_file = "./toranto_pc_latlon.csv"
df = pd.read_csv(data_file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   postal_code   103 non-null    object 
 1   borough       103 non-null    object 
 2   neighborhood  103 non-null    object 
 3   Latitude      103 non-null    float64
 4   Longitude     103 non-null    float64
dtypes: float64(2), object(3)
memory usage: 4.1+ KB


In [3]:
# sanity check for the contents of df...
df.head()

,postal_code,borough,neighborhood,Latitude,Longitude
0,M1B,North York,Parkwoods,43.806686,-79.194353
1,M1C,North York,Victoria Village,43.784535,-79.160497
2,M1E,Downtown Toronto,"Regent Park , Harbourfront",43.763573,-79.188711
3,M1G,North York,"Lawrence Manor , Lawrence Heights",43.770992,-79.216917
4,M1H,Queen,Ontario Provincial Government,43.773136,-79.239476


### Counts of borough's and neighborhood's

In [4]:
print(f"Total Borough: {df.borough.nunique()}\nTotal Neighborhood: {df.shape[0]}")

Total Borough: 15
Total Neighborhood: 103


### Map of Toronto with its neighborhood

In [6]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_exp")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [8]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['borough'], df['neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Using Foursquare API to explore the neighborhoods and segment them.

In [9]:
from configparser import ConfigParser

config = ConfigParser()
config.read("./keys.ini")

client_id = config.get("foursquare", 'ApiKey')
client_secret = config.get("foursquare", 'ApiSecret')
version = '20210628'
limit = 100 # A default Foursquare API limit value

#### Before we proceed, let's borrow the **getNearbyVenues** function from the Foursquare lab to get the venue and category

In [38]:
def getNearbyVenues(pcs, names, latitudes, longitudes, radius=500):
    
    counter = 0
    venues_list=[]
    for pc, name, lat, lng in zip(pcs, names, latitudes, longitudes):
        counter += 1
        if counter % 15 != 0:
            print(pc, end=" ")
        else:
            print(pc)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore'
        params = dict(
            client_id = client_id,
            client_secret = client_secret,
            v = version,
            ll = f"{lat}, {lng}",
            radius=radius,
            limit=limit
        )
            
        # make the GET request
        results = requests.get(url=url, params=params).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            pc,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['postal_code',
                             'neighborhood', 
                             'neighborhood_lat',  
                             'neighborhood_lon',
                             'venue', 
                             'venue_lat', 
                             'venue_lon', 
                             'venue_cat']
    
    return(nearby_venues)

In [39]:
toronto_nghd = getNearbyVenues(
    pcs = df['postal_code'],
    names=df['neighborhood'],
    latitudes=df['Latitude'],
    longitudes=df['Longitude'])

M1B M1C M1E M1G M1H M1J M1K M1L M1M M1N M1P M1R M1S M1T M1V
M1W M1X M2H M2J M2K M2L M2M M2N M2P M2R M3A M3B M3C M3H M3J
M3K M3L M3M M3N M4A M4B M4C M4E M4G M4H M4J M4K M4L M4M M4N
M4P M4R M4S M4T M4V M4W M4X M4Y M5A M5B M5C M5E M5G M5H M5J
M5K M5L M5M M5N M5P M5R M5S M5T M5V M5W M5X M6A M6B M6C M6E
M6G M6H M6J M6K M6L M6M M6N M6P M6R M6S M7A M7R M7Y M8V M8W
M8X M8Y M8Z M9A M9B M9C M9L M9M M9N M9P M9R M9V M9W 

In [40]:
# sanity check
toronto_nghd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2003 entries, 0 to 2002
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   postal_code       2003 non-null   object 
 1   neighborhood      2003 non-null   object 
 2   neighborhood_lat  2003 non-null   float64
 3   neighborhood_lon  2003 non-null   float64
 4   venue             2003 non-null   object 
 5   venue_lat         2003 non-null   float64
 6   venue_lon         2003 non-null   float64
 7   venue_cat         2003 non-null   object 
dtypes: float64(4), object(4)
memory usage: 125.3+ KB


#### count of venues for each neighborhood

In [41]:
toronto_nghd.groupby('postal_code').count()

,neighborhood,neighborhood_lat,neighborhood_lon,venue,venue_lat,venue_lon,venue_cat
postal_code,,,,,,,
M1B,2,2,2,2,2,2,2
M1C,2,2,2,2,2,2,2
M1E,9,9,9,9,9,9,9
M1G,3,3,3,3,3,3,3
M1H,9,9,9,9,9,9,9
...,...,...,...,...,...,...,...
M9N,2,2,2,2,2,2,2
M9P,9,9,9,9,9,9,9
M9R,4,4,4,4,4,4,4


#### Unique venue categories

In [42]:
print(f"There are {len(toronto_nghd['venue_cat'].unique())} uniques categories.")

There are 252 uniques categories.


### Analyze each neighborhood

In [43]:
# one hot encoding
toronto_oh = pd.get_dummies(toronto_nghd[['venue_cat']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_oh['postal_code'] = toronto_nghd['postal_code'] 

# move neighborhood column to the first column
fixed_columns = [toronto_oh.columns[-1]] + list(toronto_oh.columns[:-1])
toronto_oh = toronto_oh[fixed_columns]

toronto_oh.head()

,postal_code,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
# toronto_oh shape
toronto_oh.shape

(2003, 253)

#### group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [45]:
toronto_grpd = toronto_oh.groupby('postal_code').mean().reset_index()
toronto_grpd

,postal_code,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,M9N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,M9P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,M9R,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,M9V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
# sanity check
toronto_grpd.shape

(100, 253)

#### Pandas dataframe with neighborhood and 10 most common venues

In [22]:
# function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [58]:
# create the dataframe

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['postal_code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['postal_code'] = toronto_grpd['postal_code']

for ind in np.arange(toronto_grpd.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grpd.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,postal_code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Print Shop,Adult Boutique,Nail Salon,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark
1,M1C,Bar,Golf Course,Adult Boutique,Moroccan Restaurant,Music Store,Museum,Movie Theater,Motel,Monument / Landmark,Nail Salon
2,M1E,Mexican Restaurant,Intersection,Medical Center,Electronics Store,Donut Shop,Bank,Restaurant,Rental Car Location,Breakfast Spot,Organic Grocery
3,M1G,Coffee Shop,Korean BBQ Restaurant,Adult Boutique,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant
4,M1H,Bakery,Fried Chicken Joint,Thai Restaurant,Gas Station,Caribbean Restaurant,Athletics & Sports,Lounge,Hakka Restaurant,Bank,Moroccan Restaurant


In [59]:
neighborhoods_venues_sorted.shape

(100, 11)

### Cluster Neighborhood

Run k-means to cluster the neighborhood in 5 clusters

In [60]:
n_clust = 3

toronto_grp_clst = toronto_grpd.drop('postal_code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=n_clust, random_state=0).fit(toronto_grp_clst)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

####  Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [61]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto = df

# merge toronto_grpd with df to add latitude/longitude for each neighborhood
toronto = toronto.join(neighborhoods_venues_sorted.set_index('postal_code'), on='postal_code', how='inner')

toronto.head() # check the last columns!

,postal_code,borough,neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,North York,Parkwoods,43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Adult Boutique,Nail Salon,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark
1,M1C,North York,Victoria Village,43.784535,-79.160497,1,Bar,Golf Course,Adult Boutique,Moroccan Restaurant,Music Store,Museum,Movie Theater,Motel,Monument / Landmark,Nail Salon
2,M1E,Downtown Toronto,"Regent Park , Harbourfront",43.763573,-79.188711,1,Mexican Restaurant,Intersection,Medical Center,Electronics Store,Donut Shop,Bank,Restaurant,Rental Car Location,Breakfast Spot,Organic Grocery
3,M1G,North York,"Lawrence Manor , Lawrence Heights",43.770992,-79.216917,1,Coffee Shop,Korean BBQ Restaurant,Adult Boutique,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant
4,M1H,Queen,Ontario Provincial Government,43.773136,-79.239476,1,Bakery,Fried Chicken Joint,Thai Restaurant,Gas Station,Caribbean Restaurant,Athletics & Sports,Lounge,Hakka Restaurant,Bank,Moroccan Restaurant


In [51]:
toronto.shape

(100, 16)

In [52]:
toronto['Cluster Labels'].dtype

dtype('int32')

In [62]:
toronto['Cluster Labels'].describe()

count    100.000000
mean       0.920000
std        0.367355
min        0.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        2.000000
Name: Cluster Labels, dtype: float64

#### Visualize the resulting clusters

In [63]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(n_clust)
ys = [i + x + (i*x)**2 for i in range(n_clust)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto['Latitude'], toronto['Longitude'], toronto['neighborhood'], toronto['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Custer 1

In [64]:
toronto.loc[toronto['Cluster Labels'] == 0, toronto.columns[[1] + list(range(5, toronto.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,East York,0,Playground,Intersection,Park,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Nail Salon,Monument / Landmark
21,York,0,Park,Adult Boutique,Nail Salon,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark
23,East York,0,Park,Convenience Store,Adult Boutique,Nail Salon,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark
25,Downtown Toronto,0,Food & Drink Shop,Fast Food Restaurant,Park,Adult Boutique,Music Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant
30,Downtown Toronto,0,Park,Airport,Electronics Store,Adult Boutique,Music Store,Museum,Movie Theater,Motel,Moroccan Restaurant,Monument / Landmark
31,West Toronto,0,Grocery Store,Park,Shopping Mall,Bank,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Middle Eastern Restaurant,Music Store,Molecular Gastronomy Restaurant
40,North York,0,Park,Convenience Store,Intersection,Adult Boutique,Music Store,Museum,Movie Theater,Motel,Moroccan Restaurant,Monument / Landmark
44,Scarborough,0,Park,Lake,Bus Line,Swim School,Moroccan Restaurant,Museum,Movie Theater,Motel,Adult Boutique,Music Venue
50,North York,0,Park,Playground,Trail,Molecular Gastronomy Restaurant,Museum,Movie Theater,Motel,Moroccan Restaurant,Monument / Landmark,Modern European Restaurant
74,Central Toronto,0,Park,Women's Store,Pool,Adult Boutique,Molecular Gastronomy Restaurant,Museum,Movie Theater,Motel,Moroccan Restaurant,Monument / Landmark


cluster 1 is dominated by parks, landmarks / monuments

#### cluster 2

In [65]:
toronto.loc[toronto['Cluster Labels'] == 1, toronto.columns[[1] + list(range(5, toronto.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Fast Food Restaurant,Print Shop,Adult Boutique,Nail Salon,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark
1,North York,1,Bar,Golf Course,Adult Boutique,Moroccan Restaurant,Music Store,Museum,Movie Theater,Motel,Monument / Landmark,Nail Salon
2,Downtown Toronto,1,Mexican Restaurant,Intersection,Medical Center,Electronics Store,Donut Shop,Bank,Restaurant,Rental Car Location,Breakfast Spot,Organic Grocery
3,North York,1,Coffee Shop,Korean BBQ Restaurant,Adult Boutique,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant
4,Queen,1,Bakery,Fried Chicken Joint,Thai Restaurant,Gas Station,Caribbean Restaurant,Athletics & Sports,Lounge,Hakka Restaurant,Bank,Moroccan Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
96,Downtown Toronto,1,Pizza Place,Furniture / Home Store,Adult Boutique,Music Venue,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant
99,Downtown Toronto,1,Pizza Place,Playground,Intersection,Sandwich Place,Middle Eastern Restaurant,Discount Store,Coffee Shop,Chinese Restaurant,Park,Nail Salon
100,East TorontoBusiness reply mail Processing Centre,1,Sandwich Place,Pizza Place,Mobile Phone Shop,Bus Line,Adult Boutique,Motel,Music Store,Museum,Movie Theater,Monument / Landmark
101,Etobicoke,1,Grocery Store,Fast Food Restaurant,Coffee Shop,Beer Store,Video Store,Sandwich Place,Pharmacy,Pizza Place,Mobile Phone Shop,Modern European Restaurant


cluster 2 seems to be food heaven...

#### Cluster 3

In [66]:
toronto.loc[toronto['Cluster Labels'] == 2, toronto.columns[[1] + list(range(5, toronto.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,2,Home Service,Baseball Field,Food Truck,Adult Boutique,Moroccan Restaurant,Music Store,Museum,Movie Theater,Motel,Molecular Gastronomy Restaurant
91,Downtown Toronto,2,Construction & Landscaping,Baseball Field,Adult Boutique,Nail Salon,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark
97,Downtown Toronto,2,Baseball Field,Adult Boutique,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark


cluster 3 seems to have at least baseball field...